In [1]:
# grid search sarima hyperparameters for monthly mean sales 
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
# one-step sarima forecast
def sarima_forecast(history, config):
    order, sorder, trend = config
    # define model
    model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = sarima_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (cfg, result))    # print(' > Model[%s] %.3f' % (key, result))  -- revised from
    return (cfg, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
    scores = None
    scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
    models = list()
    # define config lists
    p_params = [0, 1, 2] # [0, 1, 2]
    d_params = [0, 1] # [0, 1]
    q_params = [0, 1, 2] # [0, 1, 2]
    t_params = ['c','t','ct']   # ['n','c','t','ct']
    P_params = [0, 1, 2] # [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2] # [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [3]:
# load dataset
df = pd.read_csv('../data/orderproducts_top20.csv', parse_dates=[1], infer_datetime_format=True)
prod_monthly = pd.crosstab(df['order_date'], df['product_sku']).resample('M').sum()
prod_monthly = prod_monthly['2018':'2020']
data = prod_monthly['EFX-FLY-BLK'].values

#### Find the optimal p,d,q,P,D,Q,t for SARIMA Modelling

In [4]:
# data split
n_test = 12
# model configs
cfg_list = sarima_configs(seasonal=[12])  # 12 months
# grid search
scores = grid_search(data, cfg_list, n_test)
# list top 10 configs
for cfg, error in scores[:10]:
    print(cfg, error)

pdq = pd.DataFrame(scores, columns=['pdq_config', 'rmse'])
pdq.to_csv('../data/pqd_scores.csv')

 > Model[[(0, 0, 0), (0, 0, 0, 12), 'c']] 11.001
 > Model[[(0, 0, 0), (0, 0, 1, 12), 'c']] 5.229
 > Model[[(0, 0, 0), (0, 0, 2, 12), 'c']] 5.541
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'c']] 6.298
 > Model[[(0, 0, 0), (0, 1, 1, 12), 'c']] 4.793
 > Model[[(0, 0, 0), (0, 1, 2, 12), 'c']] 5.858
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'c']] 4.506
 > Model[[(0, 0, 0), (1, 0, 1, 12), 'c']] 4.473
 > Model[[(0, 0, 0), (1, 0, 2, 12), 'c']] 6.047
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'c']] 4.416
 > Model[[(0, 0, 0), (1, 1, 1, 12), 'c']] 5.852
 > Model[[(0, 0, 0), (1, 1, 2, 12), 'c']] 5.858
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'c']] 8.568
 > Model[[(0, 0, 0), (2, 0, 1, 12), 'c']] 8.551
 > Model[[(0, 0, 0), (2, 0, 2, 12), 'c']] 6.985
 > Model[[(0, 0, 0), (2, 1, 0, 12), 'c']] 5.858
 > Model[[(0, 0, 0), (2, 1, 1, 12), 'c']] 5.858
 > Model[[(0, 0, 0), (2, 1, 2, 12), 'c']] 5.858
 > Model[[(0, 0, 0), (0, 0, 0, 12), 't']] 12.114
 > Model[[(0, 0, 0), (0, 0, 1, 12), 't']] 6.697
 > Model[[(0, 0, 0), (0, 0, 2, 12), 't

 > Model[[(0, 1, 0), (1, 0, 1, 12), 'c']] 4.895
 > Model[[(0, 1, 0), (1, 0, 2, 12), 'c']] 5.131
 > Model[[(0, 1, 0), (1, 1, 0, 12), 'c']] 8.927
 > Model[[(0, 1, 0), (1, 1, 1, 12), 'c']] 8.000
 > Model[[(0, 1, 0), (1, 1, 2, 12), 'c']] 4.208
 > Model[[(0, 1, 0), (2, 0, 0, 12), 'c']] 6.953
 > Model[[(0, 1, 0), (2, 0, 1, 12), 'c']] 6.953
 > Model[[(0, 1, 0), (2, 0, 2, 12), 'c']] 5.219
 > Model[[(0, 1, 0), (2, 1, 0, 12), 'c']] 4.208
 > Model[[(0, 1, 0), (2, 1, 1, 12), 'c']] 4.208
 > Model[[(0, 1, 0), (2, 1, 2, 12), 'c']] 4.208
 > Model[[(0, 1, 0), (0, 0, 0, 12), 't']] 4.990
 > Model[[(0, 1, 0), (0, 0, 1, 12), 't']] 4.982
 > Model[[(0, 1, 0), (0, 0, 2, 12), 't']] 5.271
 > Model[[(0, 1, 0), (0, 1, 0, 12), 't']] 4.335
 > Model[[(0, 1, 0), (0, 1, 1, 12), 't']] 6.795
 > Model[[(0, 1, 0), (0, 1, 2, 12), 't']] 4.923
 > Model[[(0, 1, 0), (1, 0, 0, 12), 't']] 4.940
 > Model[[(0, 1, 0), (1, 0, 1, 12), 't']] 5.041
 > Model[[(0, 1, 0), (1, 0, 2, 12), 't']] 5.202
 > Model[[(0, 1, 0), (1, 1, 0, 12), 't']

 > Model[[(1, 0, 0), (2, 0, 2, 12), 'c']] 6.677
 > Model[[(1, 0, 0), (2, 1, 0, 12), 'c']] 4.832
 > Model[[(1, 0, 0), (2, 1, 1, 12), 'c']] 4.832
 > Model[[(1, 0, 0), (2, 1, 2, 12), 'c']] 4.832
 > Model[[(1, 0, 0), (0, 0, 0, 12), 't']] 4.923
 > Model[[(1, 0, 0), (0, 0, 1, 12), 't']] 5.687
 > Model[[(1, 0, 0), (0, 0, 2, 12), 't']] 5.276
 > Model[[(1, 0, 0), (0, 1, 0, 12), 't']] 5.064
 > Model[[(1, 0, 0), (0, 1, 1, 12), 't']] 9.306
 > Model[[(1, 0, 0), (0, 1, 2, 12), 't']] 6.139
 > Model[[(1, 0, 0), (1, 0, 0, 12), 't']] 4.583
 > Model[[(1, 0, 0), (1, 0, 1, 12), 't']] 4.883
 > Model[[(1, 0, 0), (1, 0, 2, 12), 't']] 4.948
 > Model[[(1, 0, 0), (1, 1, 0, 12), 't']] 12.120
 > Model[[(1, 0, 0), (1, 1, 1, 12), 't']] 9.083
 > Model[[(1, 0, 0), (1, 1, 2, 12), 't']] 6.139
 > Model[[(1, 0, 0), (2, 0, 0, 12), 't']] 7.428
 > Model[[(1, 0, 0), (2, 0, 1, 12), 't']] 6.344
 > Model[[(1, 0, 0), (2, 0, 2, 12), 't']] 6.344
 > Model[[(1, 0, 0), (2, 1, 0, 12), 't']] 6.139
 > Model[[(1, 0, 0), (2, 1, 1, 12), 't'

 > Model[[(1, 1, 0), (0, 1, 0, 12), 't']] 4.213
 > Model[[(1, 1, 0), (0, 1, 1, 12), 't']] 7.205
 > Model[[(1, 1, 0), (0, 1, 2, 12), 't']] 5.784
 > Model[[(1, 1, 0), (1, 0, 0, 12), 't']] 4.435
 > Model[[(1, 1, 0), (1, 0, 1, 12), 't']] 4.548
 > Model[[(1, 1, 0), (1, 0, 2, 12), 't']] 5.976
 > Model[[(1, 1, 0), (1, 1, 0, 12), 't']] 8.635
 > Model[[(1, 1, 0), (1, 1, 1, 12), 't']] 7.361
 > Model[[(1, 1, 0), (1, 1, 2, 12), 't']] 5.784
 > Model[[(1, 1, 0), (2, 0, 0, 12), 't']] 5.799
 > Model[[(1, 1, 0), (2, 0, 1, 12), 't']] 10.709
 > Model[[(1, 1, 0), (2, 0, 2, 12), 't']] 10.709
 > Model[[(1, 1, 0), (2, 1, 0, 12), 't']] 5.784
 > Model[[(1, 1, 0), (2, 1, 1, 12), 't']] 5.784
 > Model[[(1, 1, 0), (2, 1, 2, 12), 't']] 5.784
 > Model[[(1, 1, 0), (0, 0, 0, 12), 'ct']] 4.112
 > Model[[(1, 1, 0), (0, 0, 1, 12), 'ct']] 5.214
 > Model[[(1, 1, 0), (0, 0, 2, 12), 'ct']] 6.346
 > Model[[(1, 1, 0), (0, 1, 0, 12), 'ct']] 5.933
 > Model[[(1, 1, 0), (0, 1, 1, 12), 'ct']] 12.352
 > Model[[(1, 1, 0), (0, 1, 2, 1

 > Model[[(2, 0, 0), (1, 0, 1, 12), 't']] 5.374
 > Model[[(2, 0, 0), (1, 0, 2, 12), 't']] 4.759
 > Model[[(2, 0, 0), (1, 1, 0, 12), 't']] 313694329390948535872683769856.000
 > Model[[(2, 0, 0), (1, 1, 1, 12), 't']] 7.510
 > Model[[(2, 0, 0), (1, 1, 2, 12), 't']] 4.480
 > Model[[(2, 0, 0), (2, 0, 0, 12), 't']] 6.597
 > Model[[(2, 0, 0), (2, 0, 1, 12), 't']] 6.207
 > Model[[(2, 0, 0), (2, 0, 2, 12), 't']] 6.178
 > Model[[(2, 0, 0), (2, 1, 0, 12), 't']] 4.480
 > Model[[(2, 0, 0), (2, 1, 1, 12), 't']] 4.480
 > Model[[(2, 0, 0), (2, 1, 2, 12), 't']] 4.480
 > Model[[(2, 0, 0), (0, 0, 0, 12), 'ct']] 4.185
 > Model[[(2, 0, 0), (0, 0, 1, 12), 'ct']] 4.465
 > Model[[(2, 0, 0), (0, 0, 2, 12), 'ct']] 5.685
 > Model[[(2, 0, 0), (0, 1, 0, 12), 'ct']] 6.756
 > Model[[(2, 0, 0), (0, 1, 1, 12), 'ct']] 18.481
 > Model[[(2, 0, 0), (0, 1, 2, 12), 'ct']] 25.804
 > Model[[(2, 0, 0), (1, 0, 0, 12), 'ct']] 5.472
 > Model[[(2, 0, 0), (1, 0, 1, 12), 'ct']] 5.365
 > Model[[(2, 0, 0), (1, 0, 2, 12), 'ct']] 8.935


 > Model[[(2, 1, 0), (1, 1, 2, 12), 't']] 7.264
 > Model[[(2, 1, 0), (2, 0, 0, 12), 't']] 28.824
 > Model[[(2, 1, 0), (2, 0, 1, 12), 't']] 8.199
 > Model[[(2, 1, 0), (2, 0, 2, 12), 't']] 8.036
 > Model[[(2, 1, 0), (2, 1, 0, 12), 't']] 7.264
 > Model[[(2, 1, 0), (2, 1, 1, 12), 't']] 7.264
 > Model[[(2, 1, 0), (2, 1, 2, 12), 't']] 7.264
 > Model[[(2, 1, 0), (0, 0, 0, 12), 'ct']] 3.943
 > Model[[(2, 1, 0), (0, 0, 1, 12), 'ct']] 5.012
 > Model[[(2, 1, 0), (0, 0, 2, 12), 'ct']] 10.078
 > Model[[(2, 1, 0), (0, 1, 0, 12), 'ct']] 6.663
 > Model[[(2, 1, 0), (0, 1, 1, 12), 'ct']] 43.500
 > Model[[(2, 1, 0), (0, 1, 2, 12), 'ct']] 16.509
 > Model[[(2, 1, 0), (1, 0, 0, 12), 'ct']] 5.609
 > Model[[(2, 1, 0), (1, 0, 1, 12), 'ct']] 5.479
 > Model[[(2, 1, 0), (1, 0, 2, 12), 'ct']] 11.155
 > Model[[(2, 1, 0), (1, 1, 0, 12), 'ct']] 17.498
 > Model[[(2, 1, 0), (1, 1, 1, 12), 'ct']] 23.008
 > Model[[(2, 1, 0), (1, 1, 2, 12), 'ct']] 16.509
 > Model[[(2, 1, 0), (2, 0, 0, 12), 'ct']] 1448224477.796
 > Model[[

In [6]:
scores[0][0]

"[(1, 0, 1), (0, 0, 0, 12), 'c']"

#### Forecast the last 3 month sales for the top 20 items

In [7]:
prod_monthly = pd.crosstab(df['order_date'], df['product_sku']).resample('M').sum()
prod_monthly = prod_monthly['2018-01':'2021-02']
items = prod_monthly.columns.values

# items are the names of the top20 items
test_predict = []
mse_list = []
results =  pd.DataFrame()
order, sorder, trend = [(1, 0, 1), (0, 0, 0, 12), 'c']   # scores[0][0]
for item in items:
    data = prod_monthly[item]
    train = data.iloc[:-3]
    test  = data.iloc[-3:]
    
    model = model = SARIMAX(train, order=order, seasonal_order=sorder, 
                            trend=trend, enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    forecast = model_fit.predict(len(train), len(train)+3)
    adj_forecast = [ 0 if x < 0 else int(round(x)) for x in forecast ]
    item_name = [item for x in range(3)]
    res = pd.DataFrame(zip(np.array(test), np.array(adj_forecast), item_name), 
                       index=['m+1','m+2','m+3'], columns=['test', 'predict', 'item'])
    results = pd.concat([results,res], axis=0)
print(results)

     test  predict             item
m+1     2        3      EFX-FLY-BLK
m+2     2        3      EFX-FLY-BLK
m+3     2        3      EFX-FLY-BLK
m+1     3        2       M80-2B-BLK
m+2     1        2       M80-2B-BLK
m+3     1        2       M80-2B-BLK
m+1     5        2       M80-2G-BLK
m+2     5        2       M80-2G-BLK
m+3     4        2       M80-2G-BLK
m+1     7        1       M80-AC-BLK
m+2     0        1       M80-AC-BLK
m+3     0        1       M80-AC-BLK
m+1     1        2       M80-AD-BLK
m+2     3        4       M80-AD-BLK
m+3     1        5       M80-AD-BLK
m+1     0        1    M80-BTY-BLK-L
m+2     3        1    M80-BTY-BLK-L
m+3     1        1    M80-BTY-BLK-L
m+1     4        1    M80-BTY-BLK-S
m+2     0        1    M80-BTY-BLK-S
m+3     2        1    M80-BTY-BLK-S
m+1     2        2       M80-EB-BLK
m+2     3        2       M80-EB-BLK
m+3     1        2       M80-EB-BLK
m+1     0        5       M80-EG-BLK
m+2     2        5       M80-EG-BLK
m+3     0        5       M80

In [9]:
results.to_csv('../data/top20predictions_last3months_v2.csv')